In [13]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [14]:
print("Length of text", len(text))

Length of text 1115394


In [15]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [16]:
# Determines the vocabulary of the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [17]:
# Creates the token vocabulary by mapping indices to chars and vice versa
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }

# Creates functions to encode and decode a string
# E.g. encode("hello") -> [1,2,3,4,5]
# E.g. decode([1,2,3,4,5]) -> "hello"
encode = lambda s: [stoi[c] for c in list(s)]
decode = lambda tokens: ''.join([itos[i] for i in tokens])

hello_tokens = encode('hello')
print(hello_tokens)
print(decode(hello_tokens))

[46, 43, 50, 50, 53]
hello


In [18]:
# Encode the text of Shakespeare mapping from chars to tokens using pytorch
import torch
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [19]:
# Create a test and train dataset
num_train = int(len(data) * 0.9)
train_data = data[:num_train]
val_data = data[num_train:]

print(len(train_data))
print(len(val_data))

1003854
111540


In [20]:
# Chunk size of data that a model will be trained on
block_size = 8

# 8 Examples -> (n1->n2), (n1,n2->n3), ..., (n1-n8->n9)
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [21]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

# Generate the context and examples
# (x->y): (n1->n2), (n1,n2->n3), ..., (n1-n8->n9)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is: {target}")

# Important to train the Transformer how to predict contexts of length 1 to block_size

When input is tensor([18]) the target is: 47
When input is tensor([18, 47]) the target is: 56
When input is tensor([18, 47, 56]) the target is: 57
When input is tensor([18, 47, 56, 57]) the target is: 58
When input is tensor([18, 47, 56, 57, 58]) the target is: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [22]:
torch.manual_seed(1337)

# Sets up mini-batches of batch_size for parallel training
batch_size = 4 # how many batches to process in parallel
block_size = 8 # size of chunks

def get_batch(split):
    data = train_data if split == 'train' else val_data
    # generates batch_size number of random starting positions for the chunks 
    # e.g. tensor([6, 3, 0, 3])
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs', xb.shape)
print(xb)

print('targets', yb.shape)
print(yb)

print('---')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"When input is {context} the target is: {target}")

inputs torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
When input is tensor([24]) the target is: 43
When input is tensor([24, 43]) the target is: 58
When input is tensor([24, 43, 58]) the target is: 5
When input is tensor([24, 43, 58,  5]) the target is: 57
When input is tensor([24, 43, 58,  5, 57]) the target is: 1
When input is tensor([24, 43, 58,  5, 57,  1]) the target is: 46
When input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is: 43
When input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is: 39
When input is tensor([44]) the target is: 53
When input is tensor([44, 53]) the target is: 56
When input is tensor([44, 53, 56]) the ta

In [23]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

"""
BigramLanguageModel that predicts the next char based on which char comes before
"""
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()

        # Creates embedding table e.g. [[1: 1,2,...65], [2: 1,2,...65], ..., [65: 1-65])
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) #(B, T, C)

        if targets is None:
            loss = None
        else:
            # pytorch wants a (B, C, T) tensor for cross_entropy
            B, T, C = logits.shape
            logits = logits.view(B*T, C)

            targets = targets.view(B*T)

            # Measures how well we are predicting the targets based on the logits
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get the prediction
            # self(idx) is equivalent to calling m.forward
            logits, _ = self(idx)

            # look at only the last timestep
            # logits will have size (B,T,C) so we want to look at T=-1
            logits = logits[:,-1,:] # becomes (B,C)

            # apply softmax to get probabilities 
            # dim=1 means that the last column has the classes
            probs = F.softmax(logits, dim=1)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(out.shape)

# Expecting a loss of -ln(1/65)=4.174
print(loss)



NameError: name 'out' is not defined

In [ ]:
def test_generation(max_new_tokens=100):
    # Test the generation function
    batch = (1,1) #batch of size 1 with the 1st token (new line)
    idx = torch.zeros(batch, dtype=torch.long)
    generated_tokens = m.generate(idx, max_new_tokens)

    print(generated_tokens)

    generated_chars = decode(generated_tokens[0].tolist())
    print(generated_chars)

test_generation()

tensor([[ 0,  5, 50, 50, 50, 50,  1, 57, 58,  1, 58, 39,  1, 42, 10,  0, 17, 24,
         21, 31,  1, 51, 43,  1, 46, 59, 56, 44,  1, 50, 39, 50,  1, 63,  6,  1,
         51, 39,  1, 42, 59, 57,  1, 54, 43,  1, 39, 58, 46, 53, 59, 53,  0, 14,
         17, 37, 10,  2,  1, 21, 52, 42, 63, 11,  1, 40, 63,  1, 57,  1, 39, 44,
         56, 43, 39, 52, 53, 53,  1, 39, 42, 47, 41, 43, 56, 43, 56, 59, 54, 39,
          1, 39, 52, 57, 43,  1, 58, 43, 41, 53, 56]])

'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecor


In [ ]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32

for steps in range(10000):
    # gets sample batches
    xb, yb = get_batch('train')    

    # evaluates the loss
    logits, loss = m(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


print(loss.item())

2.382369041442871


In [ ]:
test_generation(1000)

tensor([[ 0, 28, 50,  ..., 56, 43, 39]])

Pl llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th likl's amen, tofr,
n s Byo tred ceathe, il ivilde w
O ff y
Fivede? ig aiMy, I ivis muofounce herevern outh f athawendesees yof th withS:

FiFLINR:

Wheader y blitow,
Ye m o ditoshyd me, ch rte u hart ararwsa
Wou fe,
INurathoune
IESSARin,
MIOLened sus;
Wh.
S:
NMy BAnind g.
iudshank
An chin is a arokisupxaseru t w ity merwo al LOLo bebte loolld worinero ya l aknge ond thal ttry b's mo ge ck.

gh, inketilllin trk$nutud t ar,
WAnt cithap's Zimponcrdistherdrtes saure ' erpoperrposthel?
Handis of hef thep: ct
Ywit harfoul'st, ar izlor t ct.
Fo, sther:
I d tre th,-she.
Wowltothedl:
NNONANRI, aft,
STo way!
TIV:
WDUKI HANENEThe d ndean-bros g qpl mout fok yolaime do myoulato,
Mok h$ay t nch sle bionhoured whaneables mye.
For f beng tho; ar!TCald? min, wherur 

In [ ]:
torch.manual_seed(1337)

# batch, time, and channel
B,T,C = 4,8,2
x = torch.randn(B,T,C)
print(x.shape)

torch.Size([4, 8, 2])


tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [28]:
# bag of words
xbow = torch.zeros((B, T, C))

# For a given token at t, creates a running average all previous tokens including t
# E.g. 
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# Look at embeddings for batch 0 with 8 timesteps and 2 classes
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [ ]:
xbow[0]

# t_0 - same
# t_1 - average of t_0 + t_1 = [0.1808 - 0.3596/2=-0.0894, -0.0700 - 0.9152/2=-0.4926]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

## Matrix Multiplication can be very efficient in computing averages

In [ ]:
torch.manual_seed(42)

a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()

# First row of a \dotproduct first column of b
c = a @ b


print('a=', a)
print('b=', b)
print('c=', c)

# c[0,0] = 1*2 + 1*6 + 1*6 = 14
# c[0,1] = 1*7 + 1*4 + 1*5 = 16


a= tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c= tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
b = torch.randint(0,10,(3,2)).float()

# First row of a \dotproduct first column of b
c = a @ b


print('a=', a)
print('b=', b)
print('c=', c)

# torch.tril will create a mask
# c[0,0] = 1*2
# c[0,1] = 1*7
# c[1,0] = 1*2 + 1*6 = 8

a= tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c= tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))

# Creates the average weighting
# e.g. row one is 100%, 0%, 0%
# row two is 50%, 50%, 0%
# row three is 33%, 33%, 33%
a = a / torch.sum(a, 1, keepdim=True)


b = torch.randint(0,10,(3,2)).float()

# First row of a \dotproduct first column of b
c = a @ b


print('a=', a)
print('b=', b)
print('c=', c)

# torch.tril will create a mask
# c[0,0] = 1*2
# c[0,1] = 1*7
# c[1,0] = 1*2 + 1*6 = 8

a= tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c= tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# weighted average of each element in the row
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [41]:
# (T, T) @ (B, T, C) -> pytorch creates a (B,T,T) matrix
# (B,T,T) @ (B,T,C) -> (B,T,C)
xbow2 = wei @ x 

In [ ]:
# Version 3 - uses softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T, T))
# make everywhere with 0 negative infinity to cancel out softmax in these area
wei = wei.masked_fill(tril == 0, float('-inf'))

# will exponentiate every element and then divide by sum -> same as mean
wei = F.softmax(wei, dim = -1) # -1 is every single row

xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True